In [ ]:
# %% [markdown]
# # Análisis de Portátiles - Precios y Especificaciones
# 
# ## Introducción
# Este análisis explora las relaciones entre precios, especificaciones técnicas y ratings de portátiles para identificar patrones del mercado.

# %%
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FuncFormatter

# Configuración inicial
pd.set_option('display.max_columns', None)
sns.set_style("whitegrid")
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

# %%
# Función para formatear precios
def rupee_formatter(x, pos):
    return '₹{:,.0f}'.format(x)

# %% [markdown]
# ## 1. Carga de Datos

# %%
# Cargar datos
try:
    df = pd.read_csv("../data/laptop.csv")
    print("✅ Dataset cargado correctamente. Forma:", df.shape)
    display(df.head(3))
except Exception as e:
    print(f"❌ Error al cargar el dataset: {e}")

# %% [markdown]
# ## 2. Limpieza y Transformación de Datos

# %%
# Copia del dataframe original
clean_df = df.copy()

# Limpieza de precios
clean_df['Price'] = clean_df['Price'].str.replace('₹', '').str.replace(',', '').astype(float)

# Extracción de valores numéricos
clean_df['Ram'] = clean_df['Ram'].str.extract('(\d+)').astype(float)
clean_df['SSD'] = clean_df['SSD'].str.extract('(\d+)').astype(float)

# Normalización de nombres de columnas
clean_df.columns = clean_df.columns.str.lower().str.replace(' ', '_')

# Manejo de valores faltantes
clean_df['rating'] = clean_df['rating'].fillna(0)

# Extracción de la marca
clean_df['brand'] = clean_df['model'].str.split().str[0]

# Corrección de marcas
brand_mapping = {
    'Apple': 'Apple', 'HP': 'HP', 'Lenovo': 'Lenovo', 
    'Dell': 'Dell', 'Asus': 'Asus', 'Acer': 'Acer',
    'MSI': 'MSI', 'Samsung': 'Samsung', 'Xiaomi': 'Xiaomi',
    'Honor': 'Honor', 'Huawei': 'Huawei'
}
clean_df['brand'] = clean_df['brand'].replace(brand_mapping)

# Mostrar resultados
print("\n✅ Datos limpiados:")
display(clean_df.head(3))
print("\n📊 Resumen de variables numéricas:")
display(clean_df.describe())

# %% [markdown]
# ## 3. Análisis Exploratorio

# %%
# Matriz de correlación
print("📈 Matriz de Correlación:")
corr_matrix = clean_df[['price', 'ram', 'ssd', 'rating']].corr()
display(corr_matrix)

# Heatmap de correlación
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Matriz de Correlación entre Variables')
plt.tight_layout()
plt.savefig('../reports/images/correlation_heatmap.png', dpi=300)
plt.show()

# %%
# Conteo por marca
print("\n🏷️ Top 10 marcas por cantidad de modelos:")
top_brands = clean_df['brand'].value_counts().head(10)
display(top_brands)

plt.figure(figsize=(12, 6))
sns.barplot(x=top_brands.index, y=top_brands.values)
plt.title('Top 10 Marcas por Cantidad de Modelos')
plt.xlabel('Marca')
plt.ylabel('Número de Modelos')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('../reports/images/top_brands_count.png', dpi=300)
plt.show()

# %% [markdown]
# ## 4. Visualización de Datos

# %%
# Configuración de figura
plt.figure(figsize=(16, 12))

# 1. Distribución de precios
plt.subplot(2, 2, 1)
sns.histplot(clean_df['price'], bins=30, kde=True)
plt.gca().xaxis.set_major_formatter(FuncFormatter(rupee_formatter))
plt.title('Distribución de Precios')
plt.xlabel('Precio (₹)')
plt.ylabel('Frecuencia')

# 2. Relación RAM vs Precio
plt.subplot(2, 2, 2)
sns.boxplot(x='ram', y='price', data=clean_df)
plt.gca().yaxis.set_major_formatter(FuncFormatter(rupee_formatter))
plt.title('RAM vs Precio')
plt.xlabel('RAM (GB)')
plt.ylabel('Precio (₹)')

# 3. Rating por marca
plt.subplot(2, 2, 3)
rating_df = clean_df[clean_df['rating'] > 0]
sns.boxplot(x='brand', y='rating', data=rating_df)
plt.xticks(rotation=45)
plt.title('Rating por Marca')
plt.ylim(40, 80)

# 4. Precio vs SSD
plt.subplot(2, 2, 4)
sns.scatterplot(x='ssd', y='price', data=clean_df, hue='ram', palette='viridis')
plt.gca().yaxis.set_major_formatter(FuncFormatter(rupee_formatter))
plt.title('SSD vs Precio (coloreado por RAM)')
plt.xlabel('SSD (GB)')
plt.ylabel('Precio (₹)')

plt.tight_layout()
plt.savefig('../reports/images/main_analysis_plots.png', dpi=300)
plt.show()

# %% [markdown]
# ## 5. Análisis por Categorías

# %%
# Crear categorías
clean_df['category'] = 'General'
clean_df.loc[clean_df['model'].str.contains('Gaming', case=False), 'category'] = 'Gaming'
clean_df.loc[clean_df['brand'] == 'Apple', 'category'] = 'Apple'
clean_df.loc[clean_df['model'].str.contains('Ultrabook|Thin|Slim', case=False), 'category'] = 'Ultrabook'

# Distribución de categorías
print("\n🎮 Distribución por categorías:")
display(clean_df['category'].value_counts())

# Precios por categoría
plt.figure(figsize=(12, 6))
sns.boxplot(x='category', y='price', data=clean_df)
plt.gca().yaxis.set_major_formatter(FuncFormatter(rupee_formatter))
plt.title('Distribución de Precios por Categoría')
plt.xlabel('Categoría')
plt.ylabel('Precio (₹)')
plt.tight_layout()
plt.savefig('../reports/images/price_by_category.png', dpi=300)
plt.show()

# %% [markdown]
# ## Conclusiones

# %%
# Hallazgos principales
print("""
🔍 Hallazgos Principales:

1. Relación Precio-Componentes:
   - RAM: Correlación fuerte ({:.2f}) con el precio
   - SSD: Correlación moderada ({:.2f}) con el precio

2. Marcas Destacadas:
   - Precio más alto: {}
   - Más modelos: {}

3. Categorías:
   - Portátiles gaming son {:.1f}% más caros que el promedio
   - Apple mantiene precios premium
""".format(
    corr_matrix.loc['ram', 'price'],
    corr_matrix.loc['ssd', 'price'],
    clean_df.groupby('brand')['price'].mean().idxmax(),
    clean_df['brand'].value_counts().idxmax(),
    (clean_df[clean_df['category']=='Gaming']['price'].mean() / clean_df['price'].mean() - 1)*100
))